#### 4-1) train_data.csv 파일의 학습데이터 100 개를 이용하여 decision tree (결정 트리)를 학습하시오

In [2]:
import pandas as pd 
import numpy as np

pd_data = pd.read_csv('./train_data.csv') # train_data.csv 파일을 pandas DataFrame 으로 읽어오기

features = pd_data.iloc[:, :-1] # 4 가지 특징에 대해 데이터 선택
target = pd_data.iloc[:, -1:] # 클래스 부분만 선택

In [4]:
features

,like_human,like_walk,like_box,like_sleep
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0
...,...,...,...,...
95,0,0,0,1
96,0,0,0,1
97,0,0,1,0
98,0,0,1,0


In [5]:
target

,class
0,Dog
1,Dog
2,Dog
3,Dog
4,Dog
...,...
95,Cat
96,Cat
97,Cat
98,Cat


In [10]:
# column 'like_human'에 존재하는 값, 각 값들에 대한 개수 반환
value, counts = np.unique(pd_data['like_human'], return_counts=True)
print(value, counts)

[0 1] [46 54]


In [13]:
import pandas as pd 
import numpy as np

pd_data = pd.read_csv('./train_data.csv')

features = pd_data.iloc[:100, :-1] # 100개의 데이터만 선택
target = pd_data.iloc[:100, -1:]

def entropy(y):
    # y에 있는 클래스 레이블들의 엔트로피를 계산합니다.
    value, counts = np.unique(y, return_counts=True)
    probs = counts / len(y)
    entropy = -np.sum(probs * np.log2(probs))
    return entropy

def information_gain(X, y, feature_idx):
    # feature_idx에 해당하는 특징을 사용하여 분할했을 때의 정보 이득을 계산합니다.
    parent_entropy = entropy(y)
    values, counts = np.unique(X.iloc[:, feature_idx], return_counts=True)
    child_entropy = 0
    for value, count in zip(values, counts):
        child_y = y[X.iloc[:, feature_idx] == value]
        child_entropy += count / len(X) * entropy(child_y)
    ig = parent_entropy - child_entropy
    return ig

def find_best_feature(X, y):
    # 모든 특징들 중에서 가장 정보 이득이 큰 특징을 찾아 반환합니다.
    best_feature = None
    best_ig = 0
    for i in range(X.shape[1]):
        ig = information_gain(X, y, i)
        if ig > best_ig:
            best_feature = i
            best_ig = ig
    return best_feature

def build_tree(X, y):
    # 재귀적으로 decision tree를 구성합니다.
    # Base case: all labels are the same
    if len(np.unique(y)) == 1:
        return y.iloc[0]
    # 먼저 base case를 처리하고,
    if X.shape[1] == 0:
        return np.bincount(y).argmax()
    # recursive case에서는 find_best_feature 함수를 사용하여 가장 정보 이득이 큰 특징을 찾고, 그 특징의 값을 기준으로 데이터를 분할합니다.
    best_feature = find_best_feature(X, y)
    tree = {best_feature: {}}
    values = np.unique(X.iloc[:, best_feature])
    for value in values:
        index = X.iloc[:, best_feature] == value
        subtree = build_tree(X.loc[index], y.loc[index])
        tree[best_feature][value] = subtree
    return tree

tree = build_tree(features, target)
print(tree)

{0: {0: {3: {0: {1: {0: class    Cat
Name: 97, dtype: object, 1: class    Dog
Name: 48, dtype: object}}, 1: class    Cat
Name: 50, dtype: object}}, 1: {2: {0: class    Dog
Name: 0, dtype: object, 1: {1: {0: class    Cat
Name: 86, dtype: object, 1: class    Dog
Name: 45, dtype: object}}}}}}


#### 4-2) test_data.csv 파일의 테스트 데이터 10 개에 대한 테스트 결과를 출력하시오.

In [ ]:
import pandas as pd 

pd_data = pd.read_csv('./test_data.csv')

for i in range(10):
    row = pd_data.iloc[i, :]
    x = row[:-1]
    true_label = row[-1]
    node = tree
    while isinstance(node, dict):
        feature = list(node.keys())[0]
        value = x[feature]
        node = node[feature][value]
    pred_label = node
    print(f"Test #{i} {list(x)} -> {pred_label}")